### Import Libraries

In [ ]:
# import required libraries
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics,
                        Metrics, component)
import os
import re
from pathlib import Path

from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from google.cloud import storage
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op


### YAML Parameters

In [ ]:
#tag cell with parameters
PROJECT_ID =  ''
BUCKET_NAME=''
DATASET_ID = ''
RESOURCE_BUCKET = ''
FILE_BUCKET = ''
REGION = ''
MODEL_ID = '5090'

In [ ]:
#tag cell with parameters
PROJECT_ID =  'divg-groovyhoon-pr-d2eab4'
BUCKET_NAME='divg-groovyhoon-pr-d2eab4-default'
DATASET_ID = 'call_to_retention_dataset'
RESOURCE_BUCKET = 'divg-groovyhoon-pr-d2eab4-default'
FILE_BUCKET = 'divg-groovyhoon-pr-d2eab4-default'
MODEL_ID = '5090'
MODEL_NAME = 'call_to_retention'

### Service Parameters

In [ ]:
SERVICE_TYPE = 'call_to_retention'
SERVICE_TYPE_NAME = 'call-to-retention'
TABLE_ID = 'bq_call_to_retention_targets'
REGION = "northamerica-northeast1"

### Pipeline Parameters

In [ ]:
STACK_NAME = 'call_to_retention'
TRAIN_PIPELINE_NAME_PATH = 'call_to_retention_model/training_pipeline'
PREDICT_PIPELINE_NAME_PATH = 'call_to_retention_model/predicting_pipeline'
TRAIN_PIPELINE_NAME = 'call-to-retention-train-pipeline' # Same name as pulumi.yaml
PREDICT_PIPELINE_NAME = 'call-to-retention-predict-pipeline' # Same name as pulumi.yaml
TRAIN_PIPELINE_DESCRIPTION = 'call-to-retention-train-pipeline'
PREDICT_PIPELINE_DESCRIPTION = 'call-to-retention-predict-pipeline'
PIPELINE_ROOT = f"gs://{BUCKET_NAME}"
REGION = "northamerica-northeast1"

In [ ]:
TRAINING_DATASET_TABLE_NAME = 'bq_ctr_pipeline_dataset'
TRAINING_DATASET_SP_NAME = 'bq_sp_ctr_pipeline_dataset'
SCORE_TABLE_NAME = 'bq_call_to_retention_scores'
TEMP_TABLE='temp_call_to_retention_scores'

### Import Pipeline Components

In [ ]:
# download required component files to local
prefix = f'{STACK_NAME}/{PREDICT_PIPELINE_NAME_PATH}/components/'
dl_dir = 'components/'

storage_client = storage.Client()
bucket = storage_client.bucket(RESOURCE_BUCKET)
blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
for blob in blobs: # download each file that starts with "prefix" into "dl_dir"
    if blob.name.endswith("/"):
        continue
    file_split = blob.name.split(prefix)
    file_path = f"{dl_dir}{file_split[-1]}"
    directory = "/".join(file_path.split("/")[0:-1])
    Path(directory).mkdir(parents=True, exist_ok=True)
    blob.download_to_filename(file_path) 

# import main pipeline components
from components.bq_create_dataset import bq_create_dataset
from components.preprocess import preprocess
from components.batch_prediction import batch_prediction
from components.postprocess import postprocess
from components.campaign_data_delivery import campaign_data_delivery


### Date Parameters

In [ ]:
scoringDate = date(2023, 12, 1)  
# scoringDate = date.today() - relativedelta(days=3)/

# training dates
SCORE_DATE = scoringDate.strftime('%Y%m%d')  # date.today().strftime('%Y%m%d')
SCORE_DATE_DASH = scoringDate.strftime('%Y-%m-%d')
SCORE_DATE_MINUS_6_MOS_DASH = ((scoringDate - relativedelta(months=6)).replace(day=1)).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_START_DASH = (scoringDate.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_END_DASH = ((scoringDate.replace(day=1)) - timedelta(days=1)).strftime('%Y-%m-%d')

#revert these changes after 2023-05-30
PROMO_EXPIRY_START = (scoringDate.replace(day=1) + relativedelta(months=3)).replace(day=1).strftime('%Y-%m-%d')
PROMO_EXPIRY_END = (scoringDate.replace(day=1) + relativedelta(months=4)).replace(day=1).strftime('%Y-%m-%d')

SCORE_DATE_DELTA = 0
SCORE_DATE_VAL_DELTA = 0
TICKET_DATE_WINDOW = 30  # Days of ticket data to be queried


In [ ]:
print(scoringDate) 
print(SCORE_DATE)
print(SCORE_DATE_DASH)
print(PROMO_EXPIRY_START)
print(PROMO_EXPIRY_END)

### Pipeline

In [ ]:
# library imports
from kfp.v2 import compiler
from google.cloud.aiplatform import pipeline_jobs
@dsl.pipeline(
    name=PREDICT_PIPELINE_NAME, 
    description=PREDICT_PIPELINE_DESCRIPTION
    )
def pipeline(
        project_id: str = PROJECT_ID,
        region: str = REGION,
        resource_bucket: str = RESOURCE_BUCKET,
        file_bucket: str = FILE_BUCKET
    ):
    
    import google.oauth2.credentials
    token = !gcloud auth print-access-token
    token_str = token[0]
    
    # ----- create training set --------
    bq_create_scoring_dataset_op = bq_create_dataset(score_date=SCORE_DATE_DASH,
                          score_date_delta=SCORE_DATE_DELTA,
                          project_id=PROJECT_ID,
                          dataset_id=DATASET_ID,
                          region=REGION,
                          promo_expiry_start=PROMO_EXPIRY_START, 
                          promo_expiry_end=PROMO_EXPIRY_END, 
                          v_start_date=SCORE_DATE_MINUS_6_MOS_DASH,
                          v_end_date=SCORE_DATE_LAST_MONTH_END_DASH, 
                          token = token_str)
    
    # ----- preprocessing train data --------
    preprocess_scoring_op = preprocess(
        pipeline_dataset=TRAINING_DATASET_TABLE_NAME, 
        save_data_path='gs://{}/{}_score.csv.gz'.format(FILE_BUCKET, SERVICE_TYPE),
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID
    )

    preprocess_scoring_op.set_memory_limit('32G')
    preprocess_scoring_op.set_cpu_limit('4')

    preprocess_scoring_op.after(bq_create_scoring_dataset_op)

    batch_prediction_op = batch_prediction(
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID,
        file_bucket=FILE_BUCKET,
        service_type=SERVICE_TYPE,
        score_date_dash=SCORE_DATE_DASH,
        score_table=SCORE_TABLE_NAME, 
        temp_table=TEMP_TABLE
    )
    
    batch_prediction_op.set_memory_limit('32G')
    batch_prediction_op.set_cpu_limit('4')

    batch_prediction_op.after(preprocess_scoring_op)

    postprocessing_op = postprocess(
        project_id=PROJECT_ID,
        file_bucket=FILE_BUCKET,
        dataset_id=DATASET_ID,
        service_type=SERVICE_TYPE,
        score_date_dash=SCORE_DATE_DASH,
        temp_table=TEMP_TABLE, 
        token = token_str)
    
    postprocessing_op.set_memory_limit('32G')
    postprocessing_op.set_cpu_limit('4')
    
    postprocessing_op.after(batch_prediction_op)
    
#     # ----- create training set --------
#     campaign_data_delivery_op = campaign_data_delivery(score_date=SCORE_DATE_DASH,
#                           score_date_delta=SCORE_DATE_DELTA,
#                           project_id=PROJECT_ID,
#                           dataset_id=DATASET_ID,
#                           region=REGION)

#     campaign_data_delivery_op.set_memory_limit('32G')
#     campaign_data_delivery_op.set_cpu_limit('4')

#     campaign_data_delivery_op.after(postprocessing_op)


### Run the Pipeline Job

In [ ]:
# from kfp.v2 import compiler
# from google.cloud.aiplatform import pipeline_jobs
# import json

# compiler.Compiler().compile(
#    pipeline_func=pipeline, package_path="pipeline.json"
# )

# job = pipeline_jobs.PipelineJob(
#                                    display_name=PREDICT_PIPELINE_NAME,
#                                    template_path="pipeline.json",
#                                    location=REGION,
#                                    enable_caching=False,
#                                    pipeline_root = PIPELINE_ROOT
#                                 )
# job.run(service_account = f"bilayer-sa@{PROJECT_ID}.iam.gserviceaccount.com")


In [ ]:
import google.oauth2.credentials
import json

token = !gcloud auth print-access-token
CREDENTIALS = google.oauth2.credentials.Credentials(token[0])

compiler.Compiler().compile(
   pipeline_func=pipeline, package_path="pipeline.json"
)

job = pipeline_jobs.PipelineJob(
   display_name=PREDICT_PIPELINE_NAME,
   template_path="pipeline.json",
   credentials = CREDENTIALS,
   pipeline_root = PIPELINE_ROOT,
   location=REGION,
   enable_caching=False # I encourage you to enable caching when testing as it will reduce resource use
)

job.run()